In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pandas as pd
import os

SOURCE_CSV_FILE = os.path.abspath("./datasets/rangalamahesh_bank_churn.csv")
# Khởi tạo Spark session
spark = SparkSession.builder.appName("BankChurnPrediction").getOrCreate()

# Đọc dữ liệu
df = spark.read.csv(SOURCE_CSV_FILE, header=True, inferSchema=True)

# Hiển thị schema của dữ liệu
print("Schema của dữ liệu:")
df.printSchema()

# Hiển thị một số dòng dữ liệu
print("Một số dòng dữ liệu ban đầu:")
df.show(5)

# Giả sử chúng ta sử dụng cột "CreditScore" để tạo cột mục tiêu
# Tạo cột mục tiêu dựa trên ngưỡng điểm tín dụng (ví dụ: 700)
threshold = 700
df = df.withColumn("label", (df["CreditScore"] > threshold).cast("integer"))

# Hiển thị một số dòng dữ liệu sau khi thêm cột label
print("Dữ liệu sau khi thêm cột mục tiêu (label):")
df.select("CreditScore", "label").show(5)

# Lựa chọn các cột đặc trưng
feature_cols = ["Age", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Hiển thị schema sau khi thêm cột đặc trưng (features)
print("Schema sau khi thêm cột đặc trưng (features):")
df.printSchema()

# Hiển thị một số dòng dữ liệu sau khi thêm cột features
print("Dữ liệu sau khi thêm cột đặc trưng (features):")
df.select("features", "label").show(5)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data, test_data = df.randomSplit([0.7, 0.3], seed=42)

# Hiển thị số lượng mẫu trong tập huấn luyện và tập kiểm tra
print(f"Số lượng mẫu trong tập huấn luyện: {train_data.count()}")
print(f"Số lượng mẫu trong tập kiểm tra: {test_data.count()}")

# Huấn luyện mô hình Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_data)

# Hiển thị các hệ số của mô hình
print("Các hệ số của mô hình:")
print(model.coefficients)

# Hiển thị độ chặn của mô hình
print("Độ chặn (intercept) của mô hình:")
print(model.intercept)

# Dự đoán trên tập kiểm tra
predictions = model.transform(test_data)

# Hiển thị một số dự đoán
print("Một số dự đoán:")
predictions.select("features", "label", "prediction").show(5)

# Đánh giá mô hình
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Độ chính xác của mô hình: {accuracy}")

# Hiển thị ma trận nhầm lẫn (confusion matrix)
confusion_matrix = predictions.groupBy("label", "prediction").count().toPandas()
print("Ma trận nhầm lẫn (confusion matrix):")
print(confusion_matrix)

# Dừng Spark session
spark.stop()

Schema của dữ liệu:
root
 |-- id: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: double (nullable = true)
 |-- IsActiveMember: double (nullable = true)
 |-- EstimatedSalary: double (nullable = true)

Một số dòng dữ liệu ban đầu:
+------+----------+---------+-----------+---------+------+----+------+---------+-------------+---------+--------------+---------------+
|    id|CustomerId|  Surname|CreditScore|Geography|Gender| Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|
+------+----------+---------+-----------+---------+------+----+------+---------+-------------+---------+--------------+---------------+
|165